### Model Performance Metrics

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary

import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, roc_auc_score, roc_curve, auc
from scipy import interp
import itertools
from itertools import cycle

from tqdm import tqdm
import gc

import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%matplotlib inline

In [2]:
# set hyperparams
IMG_DIR = 'spectrogram_images/'
IMG_HEIGHT = 216
IMG_WIDTH = 216
NUM_CLASSES = 7
NUM_EPOCHS = 10
BATCH_SIZE = 32
L2_LAMBDA = 0.001

In [3]:
CUR_DIR = os.getcwd()
ROOT_DIR = os.path.dirname(CUR_DIR)
SPECTO_DIR = os.path.join(ROOT_DIR, IMG_DIR)

label_dict = {'Hip':0,
              'Pop':1,
              'Vocal':2,
              'Rhythm':3,
              'Reggae':4,
              'Rock':5,
              'Techno':6,
             }

one_hot = OneHotEncoder(categories=[range(NUM_CLASSES)])

#### Get Training and Validation Plots

In [ ]:
train_val_history_path = 'fine_tuning_resnet152_pytorch_history.pkl'
pkl_dir = os.path.join(ROOT_DIR, f'pickle_files/{train_val_history_path}')

with open(pkl_dir, 'rb') as f:
    scores = pickle.load(f)
    
print(scores.keys())
scores = pd.DataFrame(scores, index=range(1,11))

In [ ]:
plt.xticks(range(1,11))
plt.plot(scores['train_loss'], marker='o', label='training_loss')
plt.plot(scores['val_loss'], marker='d', label='validation_loss')
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Training Epochs', fontsize=12)
plt.grid()
plt.legend()
plt.savefig('plots/learning-curve-loss.png')

In [ ]:
plt.xticks(range(1,11))
plt.plot(scores['train_accuracy'], marker='o', label='training_accuracy')
plt.plot(scores['val_accuracy'], marker='d', label='validation_accuracy')
plt.ylabel('Accuracy', fontsize=12)
plt.xlabel('Training Epochs', fontsize=12)
plt.grid()
plt.legend()
plt.savefig('plots/learning-curve-accuracy.png')

#### Evaluate Best Model Epoch

In [5]:
conv_base = models.resnet152(weights='DEFAULT')
in_features = conv_base.fc.in_features
conv_base.fc = torch.nn.Identity()

test_model = nn.Sequential(
  conv_base,
  nn.Flatten(),
  nn.Linear(in_features, 512),
  nn.Dropout(p=0.3),
  nn.ReLU(),
  nn.Linear(512, NUM_CLASSES),
  nn.Softmax(dim=-1)
)

In [ ]:
best_model_path = os.path.join(ROOT_DIR, 'saved_models/fine_tuning_epoch_8_59.3413.pt')
test_model.load_state_dict(torch.load(best_model_path))

# unwrap if model saved with DataParallel
if isinstance(test_model, nn.DataParallel):
    test_model = test_model.module

test_model.eval()

In [ ]:
# load test files
pkl_dir = os.path.join(ROOT_DIR, 'pickle_files/fine_tuning_resnet152_pytorch_test_files.pkl')
with open(pkl_dir, 'rb') as f:
  test_files = pickle.load(f)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, files, specto_dir, label_dict, IMG_WIDTH, IMG_HEIGHT):
        self.files = files
        self.specto_dir = specto_dir
        self.label_dict = label_dict
        self.one_hot = one_hot
        self.IMG_WIDTH = IMG_WIDTH
        self.IMG_HEIGHT = IMG_HEIGHT

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_ = self.files[idx]
        im = Image.open(self.specto_dir + file_)
        im = im.resize((self.IMG_WIDTH, self.IMG_HEIGHT), Image.Resampling.LANCZOS)
        spectogram = np.array(im) / 255.0
        
        label = file_[:-4].split('_')
        label_array = np.array([self.label_dict[label[1]]])
        label_array = label_array.reshape(1, -1)
        label_array = one_hot.fit_transform(label_array).toarray()

        return spectogram, np.array(label_array[0])
    
# set training optimizer, loss, and metrics
optimizer = optim.Adam(test_model.parameters(), lr=1e-5, weight_decay=L2_LAMBDA)
loss_function = torch.nn.functional.cross_entropy

def categorical_accuracy(output, target):
    predicted = torch.argmax(output, dim=-1)
    labels = torch.argmax(target, dim=-1)
    correct = (predicted == labels).float()
    return correct.sum() 

In [ ]:
# Make Prediction on Test set
TEST_STEPS = len(test_files)//BATCH_SIZE

test_dataset = CustomDataset(test_files, SPECTO_DIR, label_dict, IMG_WIDTH, IMG_HEIGHT)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


test_loss = 0.0
correct_test = 0
total_test = 0

preds = np.empty((0,))
pred_labels = np.empty((0,))
pred_probs = np.empty((0,len(label_dict)))

with torch.no_grad():
  for batch_idx, (inputs, targets) in tqdm(enumerate(test_loader), total=TEST_STEPS):
    # Permute the inputs to [N, C, H, W] from [N, H, W, C]
    inputs = inputs.permute(0, 3, 1, 2)
    inputs = inputs.to(device, dtype=torch.float32)
    targets = targets.to(device, dtype=torch.float32)
    outputs = test_model(inputs)  # Forward pass
    loss = loss_function(outputs, targets)  # Calculate the loss
    test_loss += loss.item()
    total_test += targets.size(0)
    correct_test += categorical_accuracy(outputs, targets)

    # for confusion matrix later
    preds = np.concatenate((preds, torch.argmax(outputs, dim=-1).flatten()), axis=0)
    pred_labels = np.concatenate((pred_labels, torch.argmax(targets, dim=-1).flatten()), axis=0)
    # for ROC later
    pred_probs = np.concatenate((pred_probs, outputs), axis=0)

# Calculate average validation loss and accuracy
avg_test_loss = test_loss / TEST_STEPS
test_accuracy = 100. * correct_test / total_test

# Print test metrics
print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')
print('Test Set F-score =  {0:.2f}'.format(f1_score(y_true=pred_labels[:len(preds)], y_pred=preds, average='macro')))


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def one_hot_encoder(true_labels, num_records, num_classes):
    temp = np.array(true_labels[:num_records])
    true_labels = np.zeros((num_records, num_classes))
    true_labels[np.arange(num_records), temp] = 1
    return true_labels

In [ ]:
plot_confusion_matrix(confusion_matrix(y_true=pred_labels[:len(preds)], y_pred=preds), 
                      classes=label_dict.keys())

In [ ]:
one_hot_true = one_hot_encoder(pred_labels, len(preds), len(label_dict))
print('ROC AUC = {0:.3f}'.format(roc_auc_score(y_true=one_hot_true, y_score=pred_probs, average='macro')))

In [ ]:
# Compute macro-average ROC curve and ROC area
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(NUM_CLASSES):
    fpr[i], tpr[i], _ = roc_curve(one_hot_true[:, i], pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])    

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(NUM_CLASSES)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(NUM_CLASSES):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= NUM_CLASSES

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["macro"], tpr["macro"],
         label='Macro-average ROC curve (Area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', lw=1)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()